In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import tensorflow_datasets as tfds
import tensorflow as tf
batch_size=64
buffer_size=20000
"""
df1=pd.read_excel('1.xlsx')
df2=pd.read_excel('3.xlsx')
df3=pd.read_excel('6.xlsx')
df4=pd.read_excel('11.xlsx')
df5=pd.read_excel('12.xlsx')
df6=pd.read_excel('13.xlsx')
df7=pd.read_excel("qa1.xlsx")
# print(df4.columns)
# print(df5.columns)
# print(df6.columns)
df_medium1=pd.concat([df4,df5,df6])
# print(df_medium1.info())
df_medium2=pd.concat([df1,df2,df3])
print(df_medium2.info())
df_medium1.drop(['Unnamed: 0'],axis=1,inplace=True)
df_medium2.drop(['Unnamed: 0'],axis=1,inplace=True)
df_medium1.rename(columns={'question':'Q','answer':'A'},inplace=True)
df_medium2.rename(columns={'MQ':'Q','SA':'A'},inplace=True)
df_medium3=pd.concat([df_medium1,df_medium2])
print(df_medium3.info())
df_final=pd.concat([df_medium3,df7])
df_final.drop(['Unnamed: 0'],axis=1,inplace=True)
"""
df=pd.read_excel('comunications.xlsx')
df2=pd.read_csv('ChatbotData.csv')
df3=pd.read_excel('웰니스_대화_스크립트_데이터셋.xlsx')
df2.drop(['label'], axis=1,inplace=True)
df3.drop(['구분'],axis=1, inplace=True)
df3.rename(columns = {'유저' : 'Q','챗봇':'A'}, inplace = True)

df1=pd.concat([df,df2])

df_final=pd.concat([df1,df3])
df_final.dropna(axis=0, inplace=True)
print(df_final.info())

df_final=df_final.sample(frac=1).reset_index(drop=True)
print(df_final)
question=[]
answers=[]
for sentence in df_final['Q']:
    sentence=re.sub(r"([?.!,])",r"\1 ",sentence)
    sentence=sentence.strip()
    question.append(sentence)
for sentence in df_final['A']:
    sentence=re.sub(r"([?.!,])",r"\1 ",sentence)
    sentence=sentence.strip()
    answers.append(sentence)

    
# 정수 인코딩 & 단어 사전 형성
tokenizer=tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(question+answers, target_vocab_size=2**13)
START_TOKEN, END_TOKEN =[tokenizer.vocab_size],[tokenizer.vocab_size+1]
VOCAB_SIZE=tokenizer.vocab_size+2
print('임의 질문 샘플 정수 인코딩: {}'.format(tokenizer.encode(question[20])))

# 문장의 최대 정수를 정해줌
MAX_LENGTH=40
def tokenize_and_filter(inputs,outputs):
    tokenized_inputs, tokenized_outputs=[],[]
    for(sentence1, sentence2) in zip(inputs,outputs):
         # 인 코딩 시작, 종료 토큰 추가 
        sentence1=START_TOKEN+tokenizer.encode(sentence1)+END_TOKEN
        sentence2=END_TOKEN+tokenizer.encode(sentence2)+END_TOKEN
        
        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)
        
        # 패딩
        tokenized_inputs=tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=MAX_LENGTH,padding='post')
        tokenized_outputs=tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
        
        return tokenized_inputs, tokenized_outputs
    
question,answers=tokenize_and_filter(question, answers)
print(question[0])

dataset=tf.data.Dataset.from_tensor_slices((
    {
        'inputs':question,
        'dec_inputs':answers[:,:-1]
    },
    {
        'outputs':answers[:,1:]
    },

))

dataset=dataset.cache()
dataset=dataset.shuffle(buffer_size)
dataset=dataset.batch(batch_size)
dataset=dataset.prefetch(tf.data.experimental.AUTOTUNE)

# 트랜스 포머 모델
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)
  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)




class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding=self.positional_encoding(position, d_model)
    def get_angles(self, position, i, d_model):
        angles=1/tf.pow(10000,(2*(i//2))/tf.cast(d_model,tf.float32))
        return position * angles
    def positional_encoding(self,position, d_model):
        angle_rads=self.get_angles(position=tf.range(position, dtype=tf.float32)[:,tf.newaxis],i=tf.range(d_model, dtype=tf.float32)[tf.newaxis,:],d_model=d_model)
        # 짝수에는 사인 함수 적용
        sines=tf.math.sin(angle_rads[:,0::2])
        # 홀수에는 코사인 함수 적용
        cosines=tf.math.cos(angle_rads[:,1::2])
        
        angle_rads=np.zeros(angle_rads.shape)
        angle_rads[:,0::2]=sines
        angle_rads[:,1::2]=cosines
        
        pos_encoding=tf.constant(angle_rads)
        pos_encoding=pos_encoding[tf.newaxis,...]
        print(pos_encoding.shape)
        return tf.cast(pos_encoding, tf.float32)
    def call(self, inputs):
        return inputs+self.pos_encoding[:,:tf.shape(inputs)[1],:]
 
def create_padding_mask(x):
    mask=tf.cast(tf.math.equal(x,0),tf.float32)
    return mask[:,tf.newaxis,tf.newaxis,:]

# 디코더의 첫번째 서브층(sublayer)에서 미래 토큰을 Mask하는 함수
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
    return tf.maximum(look_ahead_mask, padding_mask)

#encoder
def encoder(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
    for i in range(num_layers):
        outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
            dropout=dropout, name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)     

def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

      # 인코더는 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

      # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
    attention = MultiHeadAttention(
        d_model, num_heads, name="attention")({
            'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
            'mask': padding_mask # 패딩 마스크 사용
            })

      # 드롭아웃 + 잔차 연결과 층 정규화
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(inputs + attention)

      # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
    outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

      # 드롭아웃 + 잔차 연결과 층 정규화
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
        self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
        self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    # q : (batch_size, query의 문장 길이, d_model)
    # k : (batch_size, key의 문장 길이, d_model)
    # v : (batch_size, value의 문장 길이, d_model)
    # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
        concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
        outputs = self.dense(concat_attention)

        return outputs

def scaled_dot_product_attention(query, key, value, mask):
  # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
  # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
  # padding_mask : (batch_size, 1, 1, key의 문장 길이)

  # Q와 K의 곱. 어텐션 스코어 행렬.
    matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 스케일링
  # dk의 루트값으로 나눠준다.
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

  # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
  # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
    if mask is not None:
        logits += (mask * -1e9)

  # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
  # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
    attention_weights = tf.nn.softmax(logits, axis=-1)

  # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    output = tf.matmul(attention_weights, value)

    return output, attention_weights

def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
    for i in range(num_layers):
        outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
            dropout=dropout, name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 룩어헤드 마스크(첫번째 서브층)
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name="look_ahead_mask")

  # 패딩 마스크(두번째 서브층)
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
    attention1 = MultiHeadAttention(
        d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
        })

  # 잔차 연결과 층 정규화
    attention1 = tf.keras.layers.LayerNormalization(
          epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
    attention2 = MultiHeadAttention(
        d_model, num_heads, name="attention_2")(inputs={
            'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
            'mask': padding_mask # 패딩 마스크
        })

  # 드롭아웃 + 잔차 연결과 층 정규화
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
    outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

tf.keras.backend.clear_session()
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps**-1.5)

            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

learning_rate=CustomSchedule(D_MODEL)
optimizer=tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
def accuracy(y_true,y_pred):
    y_true=tf.reshape(y_true, shape=(-1,MAX_LENGTH-1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

EPOCHS=500
model.fit(dataset,epochs=EPOCHS)


def evaluate(sentence):
    sentence = preprocess_sentence(sentence)

    sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if tf.equal(predicted_id, END_TOKEN[0]):
             break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0)

def predict(sentence):
    prediction = evaluate(sentence)

    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size])

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence
  
  
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence


output=predict("오늘 기분이 좀 좋다...")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14181 entries, 0 to 5215
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       14181 non-null  object
 1   Q       14181 non-null  object
dtypes: object(2)
memory usage: 332.4+ KB
None
                                     A                       Q
0                           많이 힘든가봅니다.                     한숨만
1      정해져 있지는 않지만 길지 않은게 서로에게 좋을 거예요.                몇일정도 썸타?
2                        내 집 마련 축하드려요.              새 집 가기로 했어
3                적극적이면 오히려 더 좋을 것 같아요.  좋아하는 사람한테 적극적인 여자 별로야?
4                   혼자 준비하기는 조금 벅찰텐대요.                셀프웨딩 해볼까
...                                ...                     ...
14176     저도 그럴 때가 많아요. 지금도 긴장하고 있는걸요?         물론 많이 떨기는 하지만요.
14177                           흔들리나요.               보고 싶다고 하네
14178                 이별의 이유가 있을 테니까요.          다시 한번 붙잡아도 안되네
14179                            공개해요.         남자친구한테 핸드폰 공개해?
14180    

1/1 [==============================] - 0s 71ms/step - loss: 0.7952 - accuracy: 0.0256
Epoch 69/500
1/1 [==============================] - 0s 70ms/step - loss: 0.7874 - accuracy: 0.0513
Epoch 70/500
1/1 [==============================] - 0s 67ms/step - loss: 0.7983 - accuracy: 0.0256
Epoch 71/500
1/1 [==============================] - 0s 65ms/step - loss: 0.8010 - accuracy: 0.0513
Epoch 72/500
1/1 [==============================] - 0s 66ms/step - loss: 0.7884 - accuracy: 0.0256
Epoch 73/500
1/1 [==============================] - 0s 71ms/step - loss: 0.7794 - accuracy: 0.0256
Epoch 74/500
1/1 [==============================] - 0s 69ms/step - loss: 0.7869 - accuracy: 0.0769
Epoch 75/500
1/1 [==============================] - 0s 71ms/step - loss: 0.7758 - accuracy: 0.0769
Epoch 76/500
1/1 [==============================] - 0s 69ms/step - loss: 0.7713 - accuracy: 0.0256
Epoch 77/500
1/1 [==============================] - 0s 66ms/step - loss: 0.7678 - accuracy: 0.0769
Epoch 78/500
1/1 [=====

1/1 [==============================] - 0s 71ms/step - loss: 0.6202 - accuracy: 0.1026
Epoch 151/500
1/1 [==============================] - 0s 67ms/step - loss: 0.6285 - accuracy: 0.1026
Epoch 152/500
1/1 [==============================] - 0s 70ms/step - loss: 0.6187 - accuracy: 0.1026
Epoch 153/500
1/1 [==============================] - 0s 68ms/step - loss: 0.6241 - accuracy: 0.1026
Epoch 154/500
1/1 [==============================] - 0s 65ms/step - loss: 0.6196 - accuracy: 0.1026
Epoch 155/500
1/1 [==============================] - 0s 63ms/step - loss: 0.6168 - accuracy: 0.1026
Epoch 156/500
1/1 [==============================] - 0s 64ms/step - loss: 0.6039 - accuracy: 0.1026
Epoch 157/500
1/1 [==============================] - 0s 63ms/step - loss: 0.6028 - accuracy: 0.1026
Epoch 158/500
1/1 [==============================] - 0s 65ms/step - loss: 0.6125 - accuracy: 0.1026
Epoch 159/500
1/1 [==============================] - 0s 64ms/step - loss: 0.6098 - accuracy: 0.1026
Epoch 160/500


1/1 [==============================] - 0s 65ms/step - loss: 0.3954 - accuracy: 0.1026
Epoch 233/500
1/1 [==============================] - 0s 67ms/step - loss: 0.4037 - accuracy: 0.1026
Epoch 234/500
1/1 [==============================] - 0s 69ms/step - loss: 0.3941 - accuracy: 0.1026
Epoch 235/500
1/1 [==============================] - 0s 66ms/step - loss: 0.3942 - accuracy: 0.1026
Epoch 236/500
1/1 [==============================] - 0s 67ms/step - loss: 0.3874 - accuracy: 0.1026
Epoch 237/500
1/1 [==============================] - 0s 71ms/step - loss: 0.3859 - accuracy: 0.1026
Epoch 238/500
1/1 [==============================] - 0s 72ms/step - loss: 0.3831 - accuracy: 0.1026
Epoch 239/500
1/1 [==============================] - 0s 71ms/step - loss: 0.3873 - accuracy: 0.1026
Epoch 240/500
1/1 [==============================] - 0s 67ms/step - loss: 0.3867 - accuracy: 0.1026
Epoch 241/500
1/1 [==============================] - 0s 65ms/step - loss: 0.3748 - accuracy: 0.1026
Epoch 242/500


1/1 [==============================] - 0s 65ms/step - loss: 0.1743 - accuracy: 0.1026
Epoch 315/500
1/1 [==============================] - 0s 63ms/step - loss: 0.1690 - accuracy: 0.1026
Epoch 316/500
1/1 [==============================] - 0s 64ms/step - loss: 0.1654 - accuracy: 0.1026
Epoch 317/500
1/1 [==============================] - 0s 65ms/step - loss: 0.1665 - accuracy: 0.1026
Epoch 318/500
1/1 [==============================] - 0s 66ms/step - loss: 0.1612 - accuracy: 0.1026
Epoch 319/500
1/1 [==============================] - 0s 66ms/step - loss: 0.1585 - accuracy: 0.1026
Epoch 320/500
1/1 [==============================] - 0s 64ms/step - loss: 0.1601 - accuracy: 0.1026
Epoch 321/500
1/1 [==============================] - 0s 65ms/step - loss: 0.1522 - accuracy: 0.1026
Epoch 322/500
1/1 [==============================] - 0s 65ms/step - loss: 0.1531 - accuracy: 0.1026
Epoch 323/500
1/1 [==============================] - 0s 67ms/step - loss: 0.1504 - accuracy: 0.1026
Epoch 324/500


1/1 [==============================] - 0s 65ms/step - loss: 0.0381 - accuracy: 0.1026
Epoch 397/500
1/1 [==============================] - 0s 66ms/step - loss: 0.0368 - accuracy: 0.1026
Epoch 398/500
1/1 [==============================] - 0s 67ms/step - loss: 0.0362 - accuracy: 0.1026
Epoch 399/500
1/1 [==============================] - 0s 66ms/step - loss: 0.0354 - accuracy: 0.1026
Epoch 400/500
1/1 [==============================] - 0s 64ms/step - loss: 0.0349 - accuracy: 0.1026
Epoch 401/500
1/1 [==============================] - 0s 66ms/step - loss: 0.0352 - accuracy: 0.1026
Epoch 402/500
1/1 [==============================] - 0s 66ms/step - loss: 0.0344 - accuracy: 0.1026
Epoch 403/500
1/1 [==============================] - 0s 70ms/step - loss: 0.0342 - accuracy: 0.1026
Epoch 404/500
1/1 [==============================] - 0s 67ms/step - loss: 0.0336 - accuracy: 0.1026
Epoch 405/500
1/1 [==============================] - 0s 69ms/step - loss: 0.0327 - accuracy: 0.1026
Epoch 406/500


1/1 [==============================] - 0s 68ms/step - loss: 0.0105 - accuracy: 0.1026
Epoch 479/500
1/1 [==============================] - 0s 68ms/step - loss: 0.0101 - accuracy: 0.1026
Epoch 480/500
1/1 [==============================] - 0s 65ms/step - loss: 0.0098 - accuracy: 0.1026
Epoch 481/500
1/1 [==============================] - 0s 68ms/step - loss: 0.0099 - accuracy: 0.1026
Epoch 482/500
1/1 [==============================] - 0s 68ms/step - loss: 0.0097 - accuracy: 0.1026
Epoch 483/500
1/1 [==============================] - 0s 70ms/step - loss: 0.0097 - accuracy: 0.1026
Epoch 484/500
1/1 [==============================] - 0s 72ms/step - loss: 0.0092 - accuracy: 0.1026
Epoch 485/500
1/1 [==============================] - 0s 71ms/step - loss: 0.0092 - accuracy: 0.1026
Epoch 486/500
1/1 [==============================] - 0s 71ms/step - loss: 0.0094 - accuracy: 0.1026
Epoch 487/500
1/1 [==============================] - 0s 67ms/step - loss: 0.0091 - accuracy: 0.1026
Epoch 488/500
